In [ ]:
''' IMPORTS '''

import pandas as pd
import numpy as np
from scipy import stats
import math
import requests
from io import BytesIO
from datetime import datetime

import nfl_data_py as nfl

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.colors as cl
from plotly.subplots import make_subplots

from PIL import Image

from resources.tier_chart import tier_chart
from resources.plotly_theme import nfl_template
from resources.get_nfl_data import get_team_info, get_pbp_data
from resources.team_stats import get_team_data

pio.templates['nfl_template'] = nfl_template

In [ ]:
''' Params / Constants '''

## Params ##
PREVIEW_WEEK = 9

AWAY_TEAM = 'IND'
HOME_TEAM = 'PIT'

## Constants ##

LEAGUE_LOGO = 'https://raw.githubusercontent.com/nflverse/nflverse-pbp/master/NFL.png'
PLOT_BUFFER = 0.1

In [ ]:
''' Import Data '''

# Import
team_data = get_team_info().rename_axis(index={'team_abbr': 'team'})
pbp_data = get_pbp_data(years=[2025])

# Stats
team_offense = get_team_data(pbp_data, unit='offense')
team_offense = team_offense.merge(team_data[['team_logo_espn', 'team_wordmark']], left_index=True, right_index=True)
print(team_offense.sort_values(by='EPA / Play', ascending=False).head().to_string())

# PBP Only matchup teams
pbp_matchup_teams = pbp_data.loc[(pbp_data['week'] < PREVIEW_WEEK) &
                                 (pbp_data['posteam'].isin([HOME_TEAM,AWAY_TEAM])), :].copy()


print(team_data.shape)
print(team_data.head().to_string())
print(pbp_matchup_teams.shape)
print(pbp_matchup_teams.head().to_string())

# Pass / Rush Tendencies

In [ ]:
## Normal Game State

# Filter 1st downs to only 1st and 10
filter_df = pbp_matchup_teams.loc[~((pbp_matchup_teams['down'] == 1) & (pbp_matchup_teams['ydstogo'] != 10)),:].copy()
filter_df['Down & Distance'] = filter_df['Down & Distance'].str.replace('1st', '1st & 10')

# Only normal snaps, Q1-3 and within 2 tds
filter_df = filter_df.loc[(filter_df['Offensive Snap']) & 
                            (~filter_df['Is Special Teams Play']) &
                            (filter_df['qtr'] <= 3) & 
                            (filter_df['score_differential'] <= 14),:]

#
filter_df['Is 1st Down'] = filter_df['down'] == 1


In [ ]:
''' Overall '''

overall_gpby = filter_df.groupby(['posteam']).aggregate(
    Plays=('Offensive Snap', 'sum'),
    Pass=('pass', 'sum'),
    Rush=('rush', 'sum'),
).rename_axis(index={'posteam': 'team'})
overall_gpby['% Pass'] = overall_gpby['Pass'] / overall_gpby['Plays']
overall_gpby['% Rush'] = overall_gpby['Rush'] / overall_gpby['Plays']

print(overall_gpby)


In [ ]:
''' '''

first_gpby = filter_df.groupby(['posteam', 'Is 1st Down']).aggregate(
    Plays=('Offensive Snap', 'sum'),
    Pass=('pass', 'sum'),
    Rush=('rush', 'sum'),
).rename_axis(index={'posteam': 'team'})
first_gpby['% Pass'] = first_gpby['Pass'] / first_gpby['Plays']
first_gpby['% Rush'] = first_gpby['Rush'] / first_gpby['Plays']

print(first_gpby)

In [ ]:
''' Down / Distance '''

down_distance_gpby = filter_df.groupby(['posteam', 'down', 'Down & Distance']).aggregate(
                                                Plays=('Offensive Snap', 'sum'),
                                                Pass=('pass', 'sum'),
                                                Rush=('rush', 'sum'),
                                           ).rename_axis(index={'posteam': 'team'})

sub_totals_1 = down_distance_gpby.groupby(['team', 'down']).sum()
sub_totals_1['Down & Distance'] = 'Total'
sub_totals_1 = sub_totals_1.set_index('Down & Distance', append=True)

sub_totals_2 = down_distance_gpby.groupby('team').sum()
sub_totals_2['down'] = 'Total'
sub_totals_2['Down & Distance'] = 'Total'
sub_totals_2 = sub_totals_2.set_index('down', append=True)
sub_totals_2 = sub_totals_2.set_index('Down & Distance', append=True)

down_distance_gpby = pd.concat([down_distance_gpby, sub_totals_1, sub_totals_2]).sort_index()
down_distance_gpby['% Pass'] = down_distance_gpby['Pass'] / down_distance_gpby['Plays']
down_distance_gpby['% Rush'] = down_distance_gpby['Rush'] / down_distance_gpby['Plays']
# down_distance_gpby['% Viz'] = np.where(down_distance_gpby['% Pass'] >= .5, down_distance_gpby['% Pass'], -1*down_distance_gpby['% Rush'])
down_distance_gpby['Diff'] = down_distance_gpby['% Pass'] - down_distance_gpby['% Rush']

down_distance_gpby = down_distance_gpby.merge(team_data['team_logo_espn'], left_index=True, right_index=True)
# print(down_distance_gpby.to_string())


# Data
data = down_distance_gpby.loc[down_distance_gpby.index.get_level_values(2) != 'Total', :]

x = data['% Pass'].tolist()
y = data.index.get_level_values(2).tolist()
logos = data['team_logo_espn'].tolist()

y_order = ['1st & 10']
for down in ['2nd', '3rd', '4th']:
    for distance in ['Short', 'Medium', 'Long']:
        y_order.append(f'{down} & {distance}')

# Plot
dot_plot = px.scatter(
    x=x,
    y=y,
)

# Figure
fig = go.Figure()

for trace in dot_plot.data:
    fig.add_trace(trace)

# Logos
for i in range(len(logos)):
    fig.add_layout_image(
        source=logos[i],  # The loaded image
        xref="x",    # Reference x-coordinates to the x-axis
        yref="y",    # Reference y-coordinates to the y-axis
        x=x[i], # X-coordinate of the image's center
        y=y[i], # Y-coordinate of the image's center
        sizex=0.75,   # Width of the image in data units
        sizey=0.75,   # Height of the image in data units
        xanchor="center", # Anchor the image by its center horizontally
        yanchor="middle", # Anchor the image by its middle vertically
        layer="above", # Place image above other plot elements
        opacity=0.8
    )

fig.update_traces(marker=dict(opacity=0))

# x_ticks = [abs(xi) for xi in x]
fig.update_xaxes(
    title=f'More Run Heavy <--  % Pass  --> More Pass Heavy',
    tickformat='.0%',
    range=[-0.05, 1.05],
    dtick=.1,
)
fig.update_yaxes(
    # type='category',
    categoryorder="array", 
    categoryarray=y_order,
    autorange='reversed',
    showgrid=False,
)
fig.update_layout(
    template='nfl_template',
    title=f'Pass / Rush Tendencies by Down & Distance',#<br><sup>Week {PREVIEW_WEEK}: {AWAY_TEAM} vs. {HOME_TEAM}</sup>',
    margin=dict(t=60,l=75, b=75),
)

# Credits
fig.add_annotation(
    text=f'"Normal" game state: Qtrs 1-3, score within 14 pts<br>Figure: @clankeranalytic | Data: nfl_data_py | {datetime.today().strftime("%Y-%m-%d")}',
    showarrow=False,
    xref='paper',
    yref='paper',
    y=-0.175, 
    x=1,
    align='right'
)

fig.show()


In [ ]:
print(pbp_matchup_teams.head().to_string())

In [ ]:
pass_tendencies_gpby = pbp_matchup_teams.loc[(pbp_matchup_teams['Offensive Snap']) & 
                                           (~pbp_matchup_teams['Is Special Teams Play']) &
                                           (pbp_matchup_teams['qtr'] <=3) &
                                           (pbp_matchup_teams['score_differential'] <= 10),:].groupby(['posteam', 'down', 'ydstogo']).aggregate(
                                                Plays=('Offensive Snap', 'sum'),
                                                Pass=('pass', 'sum'),
                                                Rush=('rush', 'sum'),
                                           ).rename_axis(index={'posteam': 'team'})


print(pass_tendencies_gpby.to_string())

In [ ]:
play_types = titans_o.groupby(['down', 'play_type']).aggregate(
    Plays=('play_type', 'size'),
    Yards=('yards_gained', 'sum')
)

print(play_types.to_string())